In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd


from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE
from solution_file_processing.constants import VRE_TECHS
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log

# Initialize config with toml file
c = sfp.SolutionFilesConfig('config_archive/ukraine/UKR.toml')

[15:52:35 00:00] I:117:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20240102_155235-UKR.log.
[15:52:37 00:01] I:149:__init__ - Initialized SolutionFilesConfig for config_archive/ukraine/UKR.toml.


In [34]:
customer_load_reg_ts = c.o.node_df[(c.o.node_df.property == 'Customer Load') |
                                        (c.o.node_df.property == 'Unserved Energy')] \
    .groupby(['model'] + c.GEO_COLS + ['timestamp']) \
    .sum() \
    .value \
    .compute() \
    .unstack(level=c.GEO_COLS)


vre_av_reg_abs_ts = c.o.gen_df[(c.o.gen_df.property == 'Available Capacity') &
                                    (c.o.gen_df.Category.isin(VRE_TECHS))] \
    .groupby((['model'] + c.GEO_COLS + ['timestamp'])) \
    .sum() \
    .value \
    .compute() \
    .unstack(level=c.GEO_COLS).fillna(0)





In [37]:
load_by_reg_ts = c.o.node_df[c.o.node_df.property == 'Load'].groupby(
        ['model'] + c.GEO_COLS + ['timestamp']).sum().value.compute().unstack(
        level='timestamp').fillna(0).stack('timestamp')

pumpload_reg_ts = (c.o.node_df[(c.o.node_df.property == 'Pump Load') |
                                   (c.o.node_df.property == 'Battery Load')]
                       .groupby(['model'] + c.GEO_COLS + ['timestamp'])
                       .sum().value.rename('Storage Load')
                       .compute())

underlying_load_reg = (load_by_reg_ts - pumpload_reg_ts).rename('Underlying Load')

net_load_reg_sto_ts = (c.v.net_load_reg_ts.stack(c.GEO_COLS).reorder_levels(
        ['model'] + c.GEO_COLS + ['timestamp']) + pumpload_reg_ts).rename('Net Load')

In [43]:
c.v.line_cap_reg.droplevel(0, axis=1)

property                     Export Limit  Import Limit
model               line                               
UKR_2021_Validation CR-ER     4820.140623  -4820.140623
                    CR-NR      405.046283   -405.046283
                    CR-SWR    7432.734076  -7432.734076
                    CR-WR     2207.547170  -2207.547170
                    EOR-ER    3422.686020  -3422.686020
                    EOR-NR     810.092567   -810.092567
                    EOR-SOR   4415.094340  -4415.094340
                    ER-NR      810.092567   -810.092567
                    ER-SOR    7250.418323  -7250.418323
                    ER-SR     5630.233190  -5630.233190
                    ER-SWR     405.046283   -405.046283
                    SOR-SR     810.092567   -810.092567
                    SR-SWR     405.046283   -405.046283
                    SWR-WR    5225.186906  -5225.186906

In [33]:

vre_regs = vre_av_reg_abs_ts.columns

In [25]:
vre_av_reg_abs_ts

Region                                         CR        EOR        ER  \
Subregion                                      CR        EOR        ER   
model               timestamp                                            
UKR_2021_Validation 2021-01-01 00:00:00  0.157402  65.769693  1.640482   
                    2021-01-01 01:00:00  0.148721  62.142110  1.550000   
                    2021-01-01 02:00:00  0.156322  65.318287  1.629223   
                    2021-01-01 03:00:00  0.170713  71.331585  1.779211   
                    2021-01-01 04:00:00  0.176702  73.834069  1.841631   
...                                           ...        ...       ...   
                    2021-12-31 19:00:00  0.093041  38.876491  0.969690   
                    2021-12-31 20:00:00  0.097679  40.814471  1.018028   
                    2021-12-31 21:00:00  0.091041  38.040922  0.948848   
                    2021-12-31 22:00:00  0.087730  36.657322  0.914337   
                    2021-12-31 23:00:00  0.086143  35.994318  0.897800   

Region                                           SOR          SR NR SWR WR  
Subregion                                        SOR          SR NR SWR WR  
model               timestamp                                               
UKR_2021_Validation 2021-01-01 00:00:00  1065.152129  330.824741  0   0  0  
                    2021-01-01 01:00:00  1006.402763  312.577822  0   0  0  
                    2021-01-01 02:00:00  1057.841515  328.554143  0   0  0  
                    2021-01-01 03:00:00  1155.227982  358.801327  0   0  0  
                    2021-01-01 04:00:00  1195.756160  371.388941  0   0  0  
...                                              ...         ... ..  .. ..  
                    2021-12-31 19:00:00   629.611840  195.550634  0   0  0  
                    2021-12-31 20:00:00   660.997768  205.298764  0   0  0  
                    2021-12-31 21:00:00   616.079634  191.347677  0   0  0  
                    2021-12-31 22:00:00   593.671993  184.388105  0   0  0  
                    2021-12-31 23:00:00   582.934520  181.053162  0   0  0  

[8760 rows x 8 columns]